In [1]:
import sys,os
__script_path=os.path.abspath(globals().get('__file__','.'))
__script_dir = os.path.dirname(__script_path)
root_dir = os.path.abspath(f'{__script_dir}/..')
print(root_dir)
for lib in [root_dir][::-1]:
    if lib in sys.path:
        sys.path.remove(lib)
    sys.path.insert(0,lib)


c:\Users\PC\Data\multimodal-rag-baseline


In [2]:
from configs.config import *
from libs.common import *
from utils.mongodb_connection import *
from utils.rag_utils import *
from utils.format_utils import *
load_dotenv(find_dotenv())

c:/Users/PC/Data/multimodal-rag-baseline


c:\ProgramData\miniconda3\envs\kyanon_digital\lib\site-packages\pypdf\_crypt_providers\_cryptography.py:32: CryptographyDeprecationWarning: ARC4 has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.ARC4 and will be removed from cryptography.hazmat.primitives.ciphers.algorithms in 48.0.0.
  from cryptography.hazmat.primitives.ciphers.algorithms import AES, ARC4


True

In [3]:
mongo_uri = os.getenv('MONGO_URI')
GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')
QUOTIENTAI = os.getenv('QuotientAI')
collection = "kyanon_digital"

mongo_client = get_mongo_client(mongo_uri)
db = mongo_client['Kyanon']
collection = db['RAG']
collection_name = "kyanon_digital"


Connection to MongoDB successful


In [4]:
exps_dir

'c:/Users/PC/Data/multimodal-rag-baseline/exps'

In [5]:
##Uncomment to initialise qdrant client in memory
client = qdrant_client.QdrantClient(
   path=f"{exps_dir}/qdrant_client_memory",
)

# ##Uncomment below to connect to Qdrant Cloud
# client = qdrant_client.QdrantClient(
#     os.environ.get("QDRANT_URL"),
#     api_key=os.environ.get("QDRANT_API_KEY"),
# )

## Uncomment below to connect to local Qdrant
#client = qdrant_client.QdrantClient("http://localhost:6333")

In [6]:
from fastembed.embedding import TextEmbedding

pd.DataFrame(TextEmbedding.list_supported_models())

2025-05-26 01:13:17.338 | WARNING  | fastembed.embedding:<module>:7 - DefaultEmbedding, FlagEmbedding, JinaEmbedding are deprecated.Use from fastembed import TextEmbedding instead.


,model,sources,model_file,description,license,size_in_GB,additional_files,dim,tasks
0,BAAI/bge-base-en,"{'hf': 'Qdrant/fast-bge-base-en', 'url': 'http...",model_optimized.onnx,"Text embeddings, Unimodal (text), English, 512...",mit,0.420,[],768,{}
1,BAAI/bge-base-en-v1.5,"{'hf': 'qdrant/bge-base-en-v1.5-onnx-q', 'url'...",model_optimized.onnx,"Text embeddings, Unimodal (text), English, 512...",mit,0.210,[],768,{}
2,BAAI/bge-large-en-v1.5,"{'hf': 'qdrant/bge-large-en-v1.5-onnx', 'url':...",model.onnx,"Text embeddings, Unimodal (text), English, 512...",mit,1.200,[],1024,{}
3,BAAI/bge-small-en,"{'hf': 'Qdrant/bge-small-en', 'url': 'https://...",model_optimized.onnx,"Text embeddings, Unimodal (text), English, 512...",mit,0.130,[],384,{}
4,BAAI/bge-small-en-v1.5,"{'hf': 'qdrant/bge-small-en-v1.5-onnx-q', 'url...",model_optimized.onnx,"Text embeddings, Unimodal (text), English, 512...",mit,0.067,[],384,{}
5,BAAI/bge-small-zh-v1.5,"{'hf': 'Qdrant/bge-small-zh-v1.5', 'url': 'htt...",model_optimized.onnx,"Text embeddings, Unimodal (text), Chinese, 512...",mit,0.090,[],512,{}
6,mixedbread-ai/mxbai-embed-large-v1,"{'hf': 'mixedbread-ai/mxbai-embed-large-v1', '...",onnx/model.onnx,"Text embeddings, Unimodal (text), English, 512...",apache-2.0,0.640,[],1024,{}
7,snowflake/snowflake-arctic-embed-xs,"{'hf': 'snowflake/snowflake-arctic-embed-xs', ...",onnx/model.onnx,"Text embeddings, Unimodal (text), English, 512...",apache-2.0,0.090,[],384,{}
8,snowflake/snowflake-arctic-embed-s,"{'hf': 'snowflake/snowflake-arctic-embed-s', '...",onnx/model.onnx,"Text embeddings, Unimodal (text), English, 512...",apache-2.0,0.130,[],384,{}
9,snowflake/snowflake-arctic-embed-m,"{'hf': 'Snowflake/snowflake-arctic-embed-m', '...",onnx/model.onnx,"Text embeddings, Unimodal (text), English, 512...",apache-2.0,0.430,[],768,{}


In [7]:
client.create_collection(
    collection_name='test',
    vectors_config=VectorParams(size=768, distance=Distance.COSINE),
)
client.get_collections()


CollectionsResponse(collections=[CollectionDescription(name='kyanon_digital'), CollectionDescription(name='test')])

In [8]:

##Initilising embedding model
## Using Default Model - BAAI/bge-small-en-v1.5
#embedding_model = TextEmbedding()

## For custom model supported by Fastembed
#embedding_model = TextEmbedding(model_name="BAAI/bge-small-en", max_length=512)
embedding_model = TextEmbedding(model_name="thenlper/gte-base", max_length=768)

## Verify the chosen Embedding model
embedding_model.model_name

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/618 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

model.onnx:   0%|          | 0.00/436M [00:00<?, ?B/s]

'thenlper/gte-base'

In [9]:

# pdf_path = "content.pdf"

# tables = camelot.read_pdf(pdf_path, pages="all", flavor='lattice')
# print(f"Tổng cộng: {tables.n} bảng được tìm thấy.")

# table_strings = []

# for i, table in enumerate(tables):
#     table_str = table.df.to_markdown(index=False)  # Dễ đọc hơn với index=False
#     table_strings.append(table_str)
#     print(f"\n--- Bảng {i + 1} ---")
#     print(table_str)



In [10]:
tables_sources = json.load(open(f"{data_dir}/final_results.json"))
sources = list(tables_sources.keys())
print(f"Tổng cộng có {len(sources)} nguồn tài liệu:")
for source in sources:
    print(f"source {source} has {len(tables_sources[source])} tables")
    print("-" * 100)
    # Loop through each table in the source
    for table in tables_sources[source]:
        
        ## Mỗi 1 table có các keys là ['table_content', 'page_numbers', 'source', 'table_idx']
        ## Trong đó table_content là markdown của bảng
        ## page_numbers là list các page number mà bảng xuất hiện
        ## source là tên của pdf
        ## table_idx là index của bảng trong pdf
        print(table['table_content'])
        print("-" * 100)

Tổng cộng có 30 nguồn tài liệu:
source 4f37fc393094547bbbe030cecedf9a3b has 7 tables
----------------------------------------------------------------------------------------------------
# 4f37fc393094547bbbe030cecedf9a3b_table_0
## Associated Context(s) Before Table:
- Film ​[ edit ]

Cross-page table spanning pages: [4, 5]
| 0     | 1                                       | 2                       | 3                    |
|:------|:----------------------------------------|:------------------------|:---------------------|
| Year  | Title                                   | Role                    | Notes                |
| 1989  | The Little Mermaid                      | Ariel                   | [29]                 |
|       |                                         | Vanessa                 |                      |
|       |                                         | (voice)                 |                      |
| 1992  | Dance! Workout with                     | Barbie (voice)  

In [11]:
import requests
import json

def summary_fn(prompt):
    """
    Gửi prompt đến Ollama và nhận về một đoạn phản hồi hoàn chỉnh.
    
    Tham số:
        prompt (str): Câu hỏi hoặc yêu cầu bạn muốn gửi đến mô hình.

    Trả về:
        str: Phản hồi hoàn chỉnh từ mô hình.
    """
    full_response = ""
    context = f"""
    You are given a single table fragment extracted from a PDF document using OCR. This fragment may be part of a larger table that was split across multiple pages due to formatting or page layout.
    Your task is to generate a detailed and comprehensive summary of the content in this table fragment.
    The summary should clearly describe:
        The main subject or topic of the table
        Key columns and their meanings
        Important patterns, trends, or observations in the data
        Any notable values or anomalies
        Contextual information needed to understand the data
    Write the summary in 1–3 full sentences , using clear and precise language.
    If applicable, mention that this fragment appears to be part of a larger table, and include any inferred continuity from the data.
    {prompt}
    """
    url = "http://localhost:11434/api/generate"
    payload = {
        "model": "gemma:latest",
        "prompt": context,
        "stream": True
    }

    with requests.post(url, json=payload, stream=True) as response:
        for line in response.iter_lines():
            if line:
                try:
                    data = json.loads(line)
                    if 'response' in data:
                        chunk = data['response']
                        full_response += chunk
                    if data.get('done', False):
                        break
                except json.JSONDecodeError:
                    continue  

    return full_response.strip()

In [20]:
def summarize_table(pdf_path):
    summary_documents = []
    tables = camelot.read_pdf(pdf_path, pages="all", flavor='lattice')
    source = os.path.splitext(os.path.basename(pdf_path))[0]
    print(f"Tổng cộng: {tables.n} bảng được tìm thấy.")
    for i, table in enumerate(tables):
        table_str = table.df.to_markdown(index=False)  # Convert to markdown for better readability

        # print(f"\n--- Bảng {i + 1} ---")
        # print(table_str)
        summary = summary_fn(table_str)

        summary_document = LangchainDocument(
            page_content=summary,
            metadata={
                "source": source,
                "page_numbers":  [int(table.page)],
                "is_table": True,
                "source_table_idx": i
            }
        )
        summary_documents.append(summary_document)
    return summary_documents

In [21]:
def get_detail_chunks(pdf_path):
    doc = fitz.open(pdf_path)
    source = os.path.splitext(os.path.basename(pdf_path))[0]

    page_documents = []
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        text = page.get_text().strip()
        if not text:
            continue

        page_document = LangchainDocument(
            page_content=text,
            metadata={
                "source": source,
                "page_numbers": [page_num + 1], 
                "is_table": False
            }
        )
        page_documents.append(page_document)

    doc.close()

    # text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=150)
    # chunks = text_splitter.split_documents(page_documents)

    # for i, chunk in enumerate(chunks):
    #     chunk.metadata.update({
    #         "chunk_id": i,
    #         "is_table": False
    #     })

    return page_documents

In [22]:
def process_all_pdfs_in_folder(folder_path):
    all_page_documents = []
    all_summary_documents = []

    for filename in os.listdir(folder_path):
        if filename.lower().endswith(".pdf"):
            pdf_path = os.path.join(folder_path, filename)
            print(f"\n>>> Đang xử lý: {pdf_path}")
            page_docs = get_detail_chunks(pdf_path)
            all_page_documents.extend(page_docs)
            summary_docs = summarize_table(pdf_path)
            all_summary_documents.extend(summary_docs)

    return all_page_documents, all_summary_documents

In [15]:
data_dir

'c:/Users/PC/Data/multimodal-rag-baseline/data'

In [23]:
all_page_documents, all_summary_documents =process_all_pdfs_in_folder(f"{data_dir}/pdf/test")
all_page_documents, all_summary_documents

Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats



>>> Đang xử lý: c:/Users/PC/Data/multimodal-rag-baseline/data/pdf/test\0c92f65db928c431023f59603039aa1e.pdf


Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBB

Tổng cộng: 41 bảng được tìm thấy.


([Document(metadata={'source': '0c92f65db928c431023f59603039aa1e', 'page_numbers': [1], 'is_table': False}, page_content="Academy Award for Best Actress\n61 languages\nArticle\nTalk\nRead\nEdit\nView history\nTools\nAcademy Award for Best Actress\nThe 2025 recipient: Mikey Madison\nAwarded for\nBest Performance by an\nActress in a Leading Role\nCountry\nUnited States\nPresented by\nAcademy of Motion Picture\nArts and Sciences (AMPAS)\nFirst award\nMay\xa016, 1929; 95 years ago\n(for films released during the\n1927/1928 film season)\nMost recent\nwinner\nMikey Madison, Anora (2024)\nMost awards\nKatharine Hepburn (4)\nMost\nnominations\nMeryl Streep (17)\nWebsite\noscars.org\nFrom Wikipedia, the free encyclopedia\nThe Academy Award for Best Actress is an award\npresented annually by the Academy of Motion Picture Arts and\nSciences (AMPAS). It has been awarded since the 1st\nAcademy Awards to an actress who has delivered an\noutstanding performance in a leading role in a film released\nt

In [19]:
all_page_documents, all_summary_documents

([Document(metadata={'source': '0c92f65db928c431023f59603039aa1e', 'page_numbers': [1], 'is_table': False}, page_content="Academy Award for Best Actress\n61 languages\nArticle\nTalk\nRead\nEdit\nView history\nTools\nAcademy Award for Best Actress\nThe 2025 recipient: Mikey Madison\nAwarded for\nBest Performance by an\nActress in a Leading Role\nCountry\nUnited States\nPresented by\nAcademy of Motion Picture\nArts and Sciences (AMPAS)\nFirst award\nMay\xa016, 1929; 95 years ago\n(for films released during the\n1927/1928 film season)\nMost recent\nwinner\nMikey Madison, Anora (2024)\nMost awards\nKatharine Hepburn (4)\nMost\nnominations\nMeryl Streep (17)\nWebsite\noscars.org\nFrom Wikipedia, the free encyclopedia\nThe Academy Award for Best Actress is an award\npresented annually by the Academy of Motion Picture Arts and\nSciences (AMPAS). It has been awarded since the 1st\nAcademy Awards to an actress who has delivered an\noutstanding performance in a leading role in a film released\nt

In [17]:
# print(process_all_pdfs_in_folder(f"{data_dir}/pdf/"))


>>> Đang xử lý: c:/Users/PC/Data/multimodal-rag-baseline/data/pdf/0c92f65db928c431023f59603039aa1e.pdf


Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBB

Tổng cộng: 41 bảng được tìm thấy.

--- Bảng 1 ---
| 0                                 | 1   |
|:----------------------------------|:----|
| Academy Award for Best Actress    |     |
| The 2025 recipient: Mikey Madison |     |
| Awarded for                       |     |
| Best Performance by an            |     |
| Actress in a Leading Role         |     |
| Country                           |     |
| United States                     |     |
| Presented by                      |     |
| Academy of Motion Picture         |     |
| Arts and Sciences (AMPAS)         |     |
| First award                       |     |
| May 16, 1929; 95 years ago        |     |
| (for films released during the    |     |
| 1927/1928 film season)            |     |
| Most recent                       |     |
| Mikey Madison, Anora (2024)       |     |
| winner                            |     |
| Most awards                       |     |
| Katharine Hepburn (4)             |     |
| Most                    

Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBB

Tổng cộng: 4 bảng được tìm thấy.

--- Bảng 1 ---
| 0                   | 1   | 2   |
|:--------------------|:----|:----|
| Henry in Bangkok in |     |     |
| February 2013       |     |     |

--- Bảng 2 ---
| 0                        | 1   | 2   |
|:-------------------------|:----|:----|
| Henry at Esquire China's |     |     |
| MAHB Awards in November  |     |     |
| 2017                     |     |     |

--- Bảng 3 ---
| 0    | 1                              | 2                                        | 3         |
|:-----|:-------------------------------|:-----------------------------------------|:----------|
| Year | Award                          | Category                                 | Result    |
| 2005 | Canadian Royal Conservatory of | Regional Gold Medal – Level 10 violin    | Won       |
|      | Music (RCM)[14][135]           |                                          |           |
| 2007 | Jean Lumb Awards[21]           | 10th Anniversary Commemorative Award for | 

...


>>> Đang xử lý: c:/Users/PC/Data/multimodal-rag-baseline/data/pdf/c935e2902adf7040a6ffe0db0f7c11e6.pdf


Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBB

Tổng cộng: 14 bảng được tìm thấy.

--- Bảng 1 ---
| 0   | 1      | 2                       | 3   |
|:----|:-------|:------------------------|:----|
|     | Career | This article is part of |     |
|     | Family | a series about          |     |
|     | Books  | Michael Jordan          |     |
|     |        | [show]                  |     |
|     |        | [show]                  |     |
|     |        | [show]                  |     |

--- Bảng 2 ---
| 0   | 1           | 2   |
|:----|:------------|:----|
|     | [show]      |     |
|     | Filmography |     |
|     | [show]      |     |
|     | Video games |     |
|     | [show]      |     |
|     | Related     |     |
|     | V · T · E   |     |

--- Bảng 3 ---
| 0   | 1              | 2   | 3    | 4                                                               |
|:----|:---------------|:----|:-----|:----------------------------------------------------------------|
|     |                |     |      | The first Air Jordan was pro

Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBB

Tổng cộng: 16 bảng được tìm thấy.

--- Bảng 1 ---
| 0             | 1                                            | 2     |
|:--------------|:---------------------------------------------|:------|
| Full name     |                                              | (Gls) |
| Date of birth |                                              | (23)  |
| Height        |                                              | (74)  |
| Position(s)   |                                              | (184) |
| 1997–2003     |                                              | (1)   |
| Years         |                                              | (282) |
| 2003–2006     |                                              | (3)   |
| 2006–2011     |                                              | (6)   |
| 2011–2021     |                                              | (2)   |
| 2021          |                                              | (41)  |
| Total         |                                              |       |
|

In [ ]:
# langchain_docs = [
#     LangchainDocument(page_content=doc["text"], metadata={"source": doc["source"]})
#     for doc in dataset
# ]

In [ ]:
# langchain_docs_table = [
#     LangchainDocument(page_content=doc["text"], metadata={"source": doc["source"]})
#     for doc in dataset
# ]

# len(langchain_docs)

In [27]:
def add_documents(client, collection_name, documents, chunk_size, chunk_overlap, embedding_model_name):
    """
    This function adds documents to the desired Qdrant collection given the specified RAG parameters.
    """

    ## Processing each document with desired TEXT_SPLITTER_ALGO, CHUNK_SIZE, CHUNK_OVERLAP
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        add_start_index=True,
        separators=["\n\n", "\n", ".", " ", ""],
    )

    docs_processed = []
    for doc in documents:
        docs_processed += text_splitter.split_documents([doc])

    ## Processing documents to be encoded by Fastembed
    docs_contents = []
    docs_metadatas = []

    for doc in docs_processed:
        if hasattr(doc, 'page_content') and hasattr(doc, 'metadata'):
            docs_contents.append(doc.page_content)
            docs_metadatas.append(doc.metadata)
        else:
            # Handle the case where attributes are missing
            print("Warning: Some documents do not have 'page_content' or 'metadata' attributes.")

    print("processed: ", len(docs_processed))
    print("content: ", len(docs_contents))
    print("metadata: ", len(docs_metadatas))

    ## Adding documents to Qdrant using desired embedding model
    client.set_model(embedding_model_name=embedding_model_name)
    client.add(collection_name=collection_name, metadata=docs_metadatas, documents=docs_contents)


def get_documents(collection_name, query, num_documents=3):
    """
    This function retrieves the desired number of documents from the Qdrant collection given a query.
    It returns a list of the retrieved documents.
    """
    search_results = client.query(
        collection_name=collection_name,
        query_text=query,
        limit=num_documents,
    )
    results = [r.metadata["document"] for r in search_results]
    return results

## Evaluation

In [30]:
from quotientai import QuotientAI
quotient = QuotientAI(api_key=QUOTIENTAI)

In [26]:
import json

with open(f"{data_dir}/generated_qa_pairs_final.json", "r", encoding="utf-8") as f:
    eval_data = json.load(f)

eval_df = pd.DataFrame(eval_data)
eval_df.head()



,id,source,page_numbers,table_idx,question,answer,context
0,qa_1_001,c46a0e327c07d6d3a04b23f7de59b55d,"[3, 4, 5, 6]",0,"What role did she play in ""Buck Rogers in the ...",Colonel Wilma Deering,# c46a0e327c07d6d3a04b23f7de59b55d_table_0\nTh...
1,qa_1_002,c46a0e327c07d6d3a04b23f7de59b55d,"[3, 4, 5, 6]",0,List the titles of the movies/TV shows release...,\n* Nuclear Hurricane\n* The Wedding Video...,# c46a0e327c07d6d3a04b23f7de59b55d_table_0\nTh...
2,qa_1_003,c46a0e327c07d6d3a04b23f7de59b55d,"[3, 4, 5, 6]",0,List the titles where she played Colonel Wilma...,\n* 1979: Buck Rogers in the 25th Century\n*...,# c46a0e327c07d6d3a04b23f7de59b55d_table_0\nTh...
3,qa_1_004,c46a0e327c07d6d3a04b23f7de59b55d,"[3, 4, 5, 6]",0,"Compare the number of episodes in ""Star Trek C...","""Star Trek Continues"" has 2 episodes, while ""T...",# c46a0e327c07d6d3a04b23f7de59b55d_table_0\nTh...
4,qa_1_005,c46a0e327c07d6d3a04b23f7de59b55d,"[3, 4, 5, 6]",0,Which TV show has the highest number of episodes?,"""Silver Spoons"" has the most episodes with 116.",# c46a0e327c07d6d3a04b23f7de59b55d_table_0\nTh...


In [34]:
import time
import pandas as pd

def run_eval(eval_df, collection_name, recipe_id, doc_retrieval_function=get_documents, num_docs=5, path=f"{exps_dir}/eval_dataset_qdrant_questions.csv"):
    """
    Evaluate the RAG pipeline based on a dataset that already has question, answer, and table-derived context.
    """

    # Step 1: Đổi tên các cột cho phù hợp
    eval_df = eval_df.rename(columns={
        "question": "input_text",
        "answer": "ground_truth",
    })

    # Step 2: Lấy context từ retriever nếu cần
    eval_df['documents'] = eval_df.apply(lambda x: doc_retrieval_function(
        collection_name=collection_name,
        query=x['input_text'],
        num_documents=num_docs), axis=1)
    
    eval_df['retrieved_context'] = eval_df['documents'].apply(lambda docs: "\n".join(docs))
    eval_df['context'] = eval_df['retrieved_context']

    # Step 3: Lưu dataset cho evaluation
    save_cols = ['input_text', 'ground_truth', 'context']
    eval_df[save_cols].to_csv(path, index=False)

    # Step 4: Upload lên QuotientAI
    dataset = quotient.create_dataset(
        file_path=path,
        name="qdrant-questions-eval-v1",
    )

    task = quotient.create_task(
        dataset_id=dataset['id'],
        name='qdrant-questions-qa-v1',
        task_type='question_answering'
    )

    job = quotient.create_job(
        task_id=task['id'],
        recipe_id=recipe_id,
        num_fewshot_examples=0,
        limit=500,
        metric_ids=[5, 7, 8, 11, 12, 13, 50],
    )

    # Step 5: Theo dõi tiến trình thủ công
    print("🔄 Waiting for evaluation job to complete...")
    while True:
        status = quotient.get_job_status(job_id=job['id'])
        print(f"⏳ Job status: {status['status']}")
        if status['status'] in ['completed', 'failed']:
            break
        time.sleep(10)  # đợi 10s trước khi kiểm tra lại

    if status['status'] == 'failed':
        raise RuntimeError("❌ Evaluation job failed.")

    # Step 6: Lấy kết quả
    data = quotient.get_eval_results(job_id=job['id'])
    df = pd.json_normalize(data, "results")

    df.columns = df.columns.str.replace("metric.", "")
    df_stats = df[df.columns[df.columns.str.contains("metric|completion_time")]]
    df_stats.columns = df_stats.columns.str.replace("metric.", "")

    metrics = {
        'completion_time_ms': 'Completion Time (ms)',
        'chunk_relevance': 'Chunk Relevance',
        'selfcheckgpt_nli_relevance': "Context Relevance",
        'selfcheckgpt_nli': "Faithfulness",
        'rougeL_fmeasure': "ROUGE-L",
        'bert_score_f1': "BERTScore",
        'bert_sentence_similarity': "BERT Sentence Similarity",
        'completion_verbosity': "Completion Verbosity",
        'verbosity_ratio': "Verbosity Ratio",
    }

    df = df.rename(columns=metrics)
    df_stats = df_stats.rename(columns=metrics)

    # Hiển thị thống kê
    display(df_stats[metrics.values()].describe())

    return df


## Experiment

### **Experiment 1:**

We will be starting off with the following specifications:
- **Embeddings: `bge-small-en`**
- **Chunk size: `512`**
- **Chunk overlap: `64`**
- **Number of docs: `3`**
- **Language model: `Mistral-7B-Instruct`**

In [36]:
chunk_size = 512
chunk_overlap = 64
embedding_model_name = "BAAI/bge-small-en"
num_docs = 3

COLLECTION_NAME = f"test"

add_documents(client,
              collection_name=COLLECTION_NAME,
              documents=all_page_documents+ all_summary_documents,
              chunk_size=chunk_size,
              chunk_overlap=chunk_overlap,
              embedding_model_name=embedding_model_name)


processed:  187
content:  187
metadata:  187


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

model_optimized.onnx:   0%|          | 0.00/133M [00:00<?, ?B/s]

AssertionError: Collection have incompatible vector params: size=768 distance=<Distance.COSINE: 'Cosine'> hnsw_config=None quantization_config=None on_disk=None datatype=None multivector_config=None

In [37]:
# Create a recipe for the generator model and prompt template
recipe_mistral = quotient.create_recipe(
    model_id=10,
    prompt_template_id=1,
    name='mistral-7b-instruct-qa-with-rag',
    description='Mistral-7b-instruct using a prompt template that includes context.'
)
recipe_mistral

AttributeError: 'QuotientAI' object has no attribute 'create_recipe'

In [ ]:
# Now we can kick off an evaluation job
experiment_1 = run_eval(eval_df,
                        collection_name=COLLECTION_NAME,
                        doc_retrieval_function=get_documents,
                        recipe_id=recipe_mistral['id'],
                        num_docs=num_docs,
                        path=f"{COLLECTION_NAME}_{num_docs}_mistral.csv")


In [ ]:
with pd.option_context('display.max_colwidth', 0):
    display(experiment_1[['content.input_text', 'content.answer','content.documents','Chunk Relevance','Context Relevance','Faithfulness']
                ].sort_values(by='Faithfulness').head(2))

### **Experiment 2 - Changing the chunk parameters**

Keeping all other parameters constant, we changed the **chunk size** and **chunk overlap** to see if we can improve performance by providing more context.

This time, our RAG setup looks like:
- **Embeddings: `bge-small-en`**
- **Chunk size: `1024`**
- **Chunk overlap: `128`**
- **Number of docs: `3`**
- **Language model: `Mistral-7B-Instruct`**

In [ ]:
## for iteration 2 - lets modify chunk configuration
## We will start with creating seperate collection to store vectors

chunk_size = 1024
chunk_overlap = 128
embedding_model_name = "BAAI/bge-small-en"
num_docs = 3

COLLECTION_NAME = f"experiment_{chunk_size}_{chunk_overlap}_{embedding_model_name.split('/')[1]}"

add_documents(client,
              collection_name=COLLECTION_NAME,
              chunk_size=chunk_size,
              chunk_overlap=chunk_overlap,
              embedding_model_name=embedding_model_name)


In [ ]:
experiment_2 = run_eval(eval_df,
                        collection_name=COLLECTION_NAME,
                        doc_retrieval_function=get_documents,
                        recipe_id=recipe_mistral['id'],
                        num_docs=num_docs,
                        path=f"{COLLECTION_NAME}_{num_docs}_mistral.csv")

In [ ]:
exp1 = ['Experiment 1:   512,   64, 3, bge-small, Mistral'] + experiment_1[main_metrics].mean().to_list()
exp2 = ['Experiment 2: 1024, 128, 3, bge-small, Mistral'] + experiment_2[main_metrics].mean().to_list()

stats = pd.DataFrame([exp1, exp2], columns=['experiment']+main_metrics)
stats = pd.pivot_table(stats, columns='experiment')
stats = stats.loc[["Chunk Relevance","Context Relevance","Faithfulness","ROUGE-L","BERTScore","BERT Sentence Similarity"]]

ax1 = stats.plot(kind='bar',
                 figsize=(8,7),
                 width=0.7, edgecolor="w", linewidth=2,
                 ylim=[0,1.1],
                 xlabel='Metric',
                 ylabel="Score",
                 title="Qdrant Documentation Question Answering",
                 color=["#65AEEA","#CEC825","#F7693C","#D8D7CA","#7D7D6C"])

ax1.legend(loc='upper left', bbox_to_anchor=(1, 1))
ax1.tick_params(axis='x', rotation=45)

for container in ax1.containers:
  ax1.bar_label(container, fmt='%.3f', rotation=90)